In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from keras.layers import Conv2D, Activation, MaxPooling2D,Flatten,Dense
from keras.callbacks import ModelCheckpoint

In [2]:
(x_train,y_train), (x_test,y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 5s 1us/step


In [3]:
x_train = x_train.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)
x_train = x_train/255
x_test = x_test/255

In [4]:
def cnn():
    model = keras.models.Sequential()
    model.add(Conv2D(32,(3,3),input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense( 10))
    model.add(Activation('softmax'))
    
    model.compile(optimizer = 'adam',
                 loss = keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ["accuracy"])
    return model

In [5]:
model = cnn()

In [6]:
checkpointer = ModelCheckpoint(filepath='best_mnist_model.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(x_train,y_train,epochs=2, batch_size=128, verbose=1)

Epoch 1/2
469/469 [==============================] - 130s 278ms/step - loss: 0.6444 - accuracy: 0.7735
Epoch 2/2
469/469 [==============================] - 131s 280ms/step - loss: 0.4124 - accuracy: 0.8543


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0

In [8]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 11s 36ms/step - loss: 0.4173 - accuracy: 0.85090s - loss: 0.4194 - accura


[0.41734784841537476, 0.8508999943733215]

# trying kerastuner

In [13]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [23]:
def cnn_with_tuning(hyper_params):
    model = keras.models.Sequential()
    
    model.add(Conv2D(hyper_params.Int("input",min_value=32,max_value=128,step=32),(3,3),input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    for i in range(hyper_params.Int("c2_layer_n",1,4)):
        model.add(Conv2D(hyper_params.Int(f"conv2_with_{i}_units",min_value=32,max_value=128,step=32),(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense( 10))
    model.add(Activation('softmax'))
    
    model.compile(optimizer = 'adam',
                 loss = keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ["accuracy"])
    return model

In [ ]:
tuner = RandomSearch(cnn_with_tuning,objective = "val_accuracy",max_trials=2,directory='tuning_log')
tuner.search(x = x_train,
            y=y_train,
            epochs = 1,
            batch_size=64,
            validation_data=(x_test,y_test))

In [29]:
print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())
print(tuner.get_best_models()[0].summary())

{'input': 64, 'c2_layer_n': 2, 'conv2_with_0_units': 32, 'conv2_with_1_units': 96}


None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 96)      